In [4]:
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
'''
node_num, feat_dim, stat_dim, num_class, T
feat_Matrix, X_Node, X_Neis, dg_list
'''
content_path = "./cora/cora.content"
cite_path = "./cora/cora.cites"

# 读取文本内容
with open(content_path, "r") as fp:
    contents = fp.readlines()
with open(cite_path, "r") as fp:
    cites = fp.readlines()

contents = np.array([np.array(l.strip().split("\t")) for l in contents])
paper_list, feat_list, label_list = np.split(contents, [1,-1], axis=1)
paper_list, label_list = np.squeeze(paper_list), np.squeeze(label_list)
# Paper -> Index dict
paper_dict = dict([(key, val) for val, key in enumerate(paper_list)])
# Label -> Index 字典
labels = list(set(label_list))
label_dict = dict([(key, val) for val, key in enumerate(labels)])
# Edge_index
cites = [i.strip().split("\t") for i in cites]
cites = np.array([[paper_dict[i[0]], paper_dict[i[1]]] for i in cites], 
                 np.int64).T   # (2, edge)
cites = np.concatenate((cites, cites[::-1, :]), axis=1)  # (2, 2*edge) or (2, E)
# Degree
_, degree_list = np.unique(cites[0,:], return_counts=True)

# Input
node_num = len(paper_list)
feat_dim = feat_list.shape[1]
stat_dim = 32
num_class = len(labels)
T = 2
feat_Matrix = torch.Tensor(feat_list.astype(np.float32))
X_Node, X_Neis = np.split(cites, 2, axis=0)
X_Node, X_Neis = torch.from_numpy(np.squeeze(X_Node)), \
                 torch.from_numpy(np.squeeze(X_Neis))
dg_list = degree_list[X_Node]
label_list = np.array([label_dict[i] for i in label_list])
label_list = torch.from_numpy(label_list)

In [6]:
print("{}Data Process Info{}".format("*"*20, "*"*20))
print("==> Number of node : {}".format(node_num))
print("==> Number of edges : {}/2={}".format(cites.shape[1], int(cites.shape[1]/2)))
print("==> Number of classes : {}".format(num_class))
print("==> Dimension of node features : {}".format(feat_dim))
print("==> Dimension of node state : {}".format(stat_dim))
print("==> T : {}".format(T))
print("==> Shape of feat_Matrix : {}".format(feat_Matrix.shape))
print("==> Shape of X_Node : {}".format(X_Node.shape))
print("==> Shape of X_Neis : {}".format(X_Neis.shape))
print("==> Length of dg_list : {}".format(len(dg_list)))

********************Data Process Info********************
==> Number of node : 2708
==> Number of edges : 10858/2=5429
==> Number of classes : 7
==> Dimension of node features : 1433
==> Dimension of node state : 32
==> T : 2
==> Shape of feat_Matrix : torch.Size([2708, 1433])
==> Shape of X_Node : torch.Size([10858])
==> Shape of X_Neis : torch.Size([10858])
==> Length of dg_list : 10858


In [7]:
class Xi(nn.Module):
    def __init__(self, ln, s):
        super(Xi, self).__init__()
        self.ln = ln   # 节点特征向量的维度
        self.s = s     # 节点的个数
        
        # 线性网络层
        self.linear = nn.Linear(in_features=2 * ln,
                                out_features=s ** 2,
                                bias=True)
        # 激活函数
        self.tanh = nn.Tanh()
        
    def forward(self, X):
        bs = X.size()[0]
        out = self.linear(X)
        out = self.tanh(out)
        return out.view(bs, self.s, self.s)

class Rou(nn.Module):
    def __init__(self, ln, s):
        super(Rou, self).__init__()
        self.linear = nn.Linear(in_features=ln,
                                out_features=s,
                                bias=True)
        self.tanh = nn.Tanh()
    def forward(self, X):
        return self.tanh(self.linear(X))

class Hw(nn.Module):
    def __init__(self, ln, s, mu=0.9):
        super(Hw, self).__init__()
        self.ln = ln
        self.s = s
        self.mu = mu
        
        # 初始化网络层
        self.Xi = Xi(ln, s)
        self.Rou = Rou(ln, s)
    
    def forward(self, X, H, dg_list):
        if isinstance(dg_list, list) or isinstance(dg_list, np.ndarray):
            dg_list = torch.Tensor(dg_list).to(X.device)
        elif isinstance(dg_list, torch.Tensor):
            pass
        else:
            raise TypeError("==> dg_list should be list or tensor, not {}".format(type(dg_list)))
        A = (self.Xi(X) * self.mu / self.s) / dg_list.view(-1, 1, 1)# (N, S, S)
        b = self.Rou(torch.chunk(X, chunks=2, dim=1)[0])# (N, S)
        out = torch.squeeze(torch.matmul(A, torch.unsqueeze(H, 2)),-1) + b  # (N, s, s) * (N, s) + (N, s)
        return out    # (N, s)

class AggrSum(nn.Module):
    def __init__(self, node_num):
        super(AggrSum, self).__init__()
        self.V = node_num
    
    def forward(self, H, X_node):
        # H : (N, s) -> (V, s)
        # X_node : (N, )
        mask = torch.stack([X_node] * self.V, 0)
        mask = mask.float() - torch.unsqueeze(torch.range(0,self.V-1).float(), 1)
        mask = (mask == 0).float()
        # (V, N) * (N, s) -> (V, s)
        return torch.mm(mask, H)


class OriLinearGNN(nn.Module):
    def __init__(self, node_num, feat_dim, stat_dim, num_class, T):
        super(OriLinearGNN, self).__init__()
        self.embed_dim = feat_dim
        self.stat_dim = stat_dim
        self.T = T
        # 输出层
        '''
        self.out_layer = nn.Sequential(
            nn.Linear(stat_dim, 16),   # ln+s -> hidden_layer
            nn.Tanh(),
            nn.Dropout(p=0.5),
            nn.Linear(16, num_class)   # hidden_layer -> logits
        )
        '''
        self.out_layer = nn.Linear(stat_dim, num_class)
        self.dropout = nn.Dropout()
        self.log_softmax = nn.LogSoftmax(dim=-1)
        # 实现Fw
        self.Hw = Hw(feat_dim, stat_dim)
        # 实现H的分组求和
        self.Aggr = AggrSum(node_num)
        
    def forward(self, feat_Matrix, X_Node, X_Neis, dg_list):
        node_embeds = torch.index_select(input=feat_Matrix,
                                         dim=0,
                                         index=X_Node)  # (N, ln)
        neis_embeds = torch.index_select(input=feat_Matrix,
                                         dim=0,
                                         index=X_Neis)  # (N, ln)
        X = torch.cat((node_embeds, neis_embeds), 1)    # (N, 2 * ln)
        H = torch.zeros((feat_Matrix.shape[0], self.stat_dim), dtype=torch.float32)  # (V, s)
        H = H.to(feat_Matrix.device)
        # 循环T次计算
        for t in range(self.T):
            # (V, s) -> (N, s)
            H = torch.index_select(H, 0, X_Neis)
            # (N, s) -> (N, s)
            H = self.Hw(X, H, dg_list)
            # (N, s) -> (V, s)
            H = self.Aggr(H, X_Node)
            # print(H[1])
        # out = torch.cat((feat_Matrix, H), 1)   # (V, ln+s)
        out = self.log_softmax(self.dropout(self.out_layer(H)))
        return out  # (V, num_class)

In [8]:
# Split dataset
train_mask = torch.zeros(node_num, dtype=torch.uint8)
train_mask[:node_num - 1000] = 1                  # 1700左右training
val_mask = None                                    # 0valid
test_mask = torch.zeros(node_num, dtype=torch.uint8)
test_mask[node_num - 500:] = 1                    # 500test

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = OriLinearGNN(node_num, feat_dim, stat_dim, num_class, T).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-3)
feat_Matrix = feat_Matrix.to(device)
X_Node = X_Node.to(device)
X_Neis = X_Neis.to(device)

for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    
    # Get output
    out = model(feat_Matrix, X_Node, X_Neis, dg_list)
    
    # Get loss
    loss = F.nll_loss(out[train_mask], label_list[train_mask])
    _, pred = out.max(dim=1)
    
    # Get predictions and calculate training accuracy
    correct = float(pred[train_mask].eq(label_list[train_mask]).sum().item())
    acc = correct / train_mask.sum().item()
    print('[Epoch {}/200] Loss {:.4f}, train acc {:.4f}'.format(epoch, loss.cpu().detach().data.item(), acc))
    
    # Backward
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data every 10 epochs
    if (epoch+1) % 10 == 0:
        model.eval()
        _, pred = model(feat_Matrix, X_Node, X_Neis, dg_list).max(dim=1)
        correct = float(pred[test_mask].eq(label_list[test_mask]).sum().item())
        acc = correct / test_mask.sum().item()
        print('Accuracy: {:.4f}'.format(acc))

/Users/ivyone/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:116: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].


[Epoch 0/200] Loss 2.0693, train acc 0.1142
[Epoch 1/200] Loss 1.7313, train acc 0.3800
[Epoch 2/200] Loss 1.4347, train acc 0.4918
[Epoch 3/200] Loss 1.2367, train acc 0.5281
[Epoch 4/200] Loss 1.1052, train acc 0.5744
[Epoch 5/200] Loss 1.0255, train acc 0.5790
[Epoch 6/200] Loss 0.9700, train acc 0.6054
[Epoch 7/200] Loss 0.9487, train acc 0.6001
[Epoch 8/200] Loss 0.8957, train acc 0.6230
[Epoch 9/200] Loss 0.8920, train acc 0.6019
Accuracy: 0.7600
[Epoch 10/200] Loss 0.8451, train acc 0.6288
[Epoch 11/200] Loss 0.8192, train acc 0.6352
[Epoch 12/200] Loss 0.7872, train acc 0.6499
[Epoch 13/200] Loss 0.7985, train acc 0.6393
[Epoch 14/200] Loss 0.8149, train acc 0.6335
[Epoch 15/200] Loss 0.8032, train acc 0.6370
[Epoch 16/200] Loss 0.7923, train acc 0.6376
[Epoch 17/200] Loss 0.7545, train acc 0.6581
[Epoch 18/200] Loss 0.7832, train acc 0.6423
[Epoch 19/200] Loss 0.7590, train acc 0.6429
Accuracy: 0.7640
[Epoch 20/200] Loss 0.7525, train acc 0.6376
[Epoch 21/200] Loss 0.7474, tra

[Epoch 175/200] Loss 0.7039, train acc 0.6364
[Epoch 176/200] Loss 0.7229, train acc 0.6505
[Epoch 177/200] Loss 0.6950, train acc 0.6522
[Epoch 178/200] Loss 0.6876, train acc 0.6616
[Epoch 179/200] Loss 0.6963, train acc 0.6499
Accuracy: 0.7300
[Epoch 180/200] Loss 0.6808, train acc 0.6674
[Epoch 181/200] Loss 0.7255, train acc 0.6464
[Epoch 182/200] Loss 0.6827, train acc 0.6663
[Epoch 183/200] Loss 0.7208, train acc 0.6452
[Epoch 184/200] Loss 0.6974, train acc 0.6423
[Epoch 185/200] Loss 0.6904, train acc 0.6534
[Epoch 186/200] Loss 0.6961, train acc 0.6434
[Epoch 187/200] Loss 0.7071, train acc 0.6511
[Epoch 188/200] Loss 0.6924, train acc 0.6358
[Epoch 189/200] Loss 0.6977, train acc 0.6528
Accuracy: 0.7140
[Epoch 190/200] Loss 0.6926, train acc 0.6563
[Epoch 191/200] Loss 0.6875, train acc 0.6651
[Epoch 192/200] Loss 0.6760, train acc 0.6663
[Epoch 193/200] Loss 0.6824, train acc 0.6563
[Epoch 194/200] Loss 0.6579, train acc 0.6803
[Epoch 195/200] Loss 0.7136, train acc 0.6470


In [9]:
# Split dataset
train_mask = torch.zeros(node_num, dtype=torch.uint8)
train_mask[:node_num - 1000] = 1                  # 1700左右training
val_mask = None                                    # 0valid
test_mask = torch.zeros(node_num, dtype=torch.uint8)
test_mask[node_num - 500:] = 1                    # 500test
x = feat_Matrix
edge_index = torch.from_numpy(cites)

In [10]:
class AggrSum(nn.Module):
    def __init__(self, node_num):
        super(AggrSum, self).__init__()
        self.V = node_num
    
    def forward(self, H, X_node):
        # H : (N, s) -> (V, s)
        # X_node : (N, )
        mask = torch.stack([X_node] * self.V, 0)
        mask = mask.float() - torch.unsqueeze(torch.range(0,self.V-1).float(), 1)
        mask = (mask == 0).float()
        # (V, N) * (N, s) -> (V, s)
        return torch.mm(mask, H)


class GCNConv(nn.Module):
    def __init__(self, in_channel, out_channel, node_num):
        super(GCNConv, self).__init__()
        self.linear = nn.Linear(in_channel, out_channel)
        self.aggregation = AggrSum(node_num)
        
    def forward(self, x, edge_index):
        # Add self-connect edges
        edge_index = self.addSelfConnect(edge_index, x.shape[0])
        
        # Apply linear transform
        x = self.linear(x)
        
        # Normalize message
        row, col = edge_index
        deg = self.calDegree(row, x.shape[0]).float()
        deg_sqrt = deg.pow(-0.5)  # (N, )
        norm = deg_sqrt[row] * deg_sqrt[col]
        
        # Node feature matrix
        tar_matrix = torch.index_select(x, dim=0, index=col)
        tar_matrix = norm.view(-1, 1) * tar_matrix  # (E, out_channel)
        # Aggregate information
        aggr =  self.aggregation(tar_matrix, row)  # (N, out_channel)
        return aggr
        
    
    def calDegree(self, edges, num_nodes):
        ind, deg = np.unique(edges.cpu().numpy(), return_counts=True)
        deg_tensor = torch.zeros((num_nodes, ), dtype=torch.long)
        deg_tensor[ind] = torch.from_numpy(deg)
        return deg_tensor.to(edges.device)
    
    def addSelfConnect(self, edge_index, num_nodes):
        selfconn = torch.stack([torch.range(0, num_nodes-1, dtype=torch.long)]*2,
                               dim=0).to(edge_index.device)
        return torch.cat(tensors=[edge_index, selfconn],
                         dim=1)
    
        
class Net(nn.Module):
    def __init__(self, feat_dim, num_class, num_node):
        super(Net, self).__init__()
        self.conv1 = GCNConv(feat_dim, 16, num_node)
        self.conv2 = GCNConv(16, num_class, num_node)
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.log_softmax(x, dim=1)

In [11]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(feat_dim, num_class, node_num).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
x = x.to(device)
edge_index = edge_index.to(device)

for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    
    # Get output
    out = model(x, edge_index)
    
    # Get loss
    loss = F.nll_loss(out[train_mask], label_list[train_mask])
    _, pred = out.max(dim=1)
    
    # Get predictions and calculate training accuracy
    correct = float(pred[train_mask].eq(label_list[train_mask]).sum().item())
    acc = correct / train_mask.sum().item()
    print('[Epoch {}/200] Loss {:.4f}, train acc {:.4f}'.format(epoch, loss.cpu().detach().data.item(), acc))
    
    # Backward
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data every 10 epochs
    if (epoch+1) % 10 == 0:
        model.eval()
        _, pred = model(x, edge_index).max(dim=1)
        correct = float(pred[test_mask].eq(label_list[test_mask]).sum().item())
        acc = correct / test_mask.sum().item()
        print('Accuracy: {:.4f}'.format(acc))

/Users/ivyone/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:65: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
/Users/ivyone/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  del sys.path[0]


[Epoch 0/200] Loss 1.9893, train acc 0.0632
[Epoch 1/200] Loss 1.9549, train acc 0.0989
[Epoch 2/200] Loss 1.9022, train acc 0.2652
[Epoch 3/200] Loss 1.8434, train acc 0.4344
[Epoch 4/200] Loss 1.7754, train acc 0.4906
[Epoch 5/200] Loss 1.7042, train acc 0.5047
[Epoch 6/200] Loss 1.6479, train acc 0.4795
[Epoch 7/200] Loss 1.5631, train acc 0.4783
[Epoch 8/200] Loss 1.5176, train acc 0.4666
[Epoch 9/200] Loss 1.4271, train acc 0.5293
Accuracy: 0.4740
[Epoch 10/200] Loss 1.3615, train acc 0.5398
[Epoch 11/200] Loss 1.2943, train acc 0.5966
[Epoch 12/200] Loss 1.2342, train acc 0.6587
[Epoch 13/200] Loss 1.1590, train acc 0.6821
[Epoch 14/200] Loss 1.1094, train acc 0.7272
[Epoch 15/200] Loss 1.0289, train acc 0.7617
[Epoch 16/200] Loss 0.9938, train acc 0.7740
[Epoch 17/200] Loss 0.9277, train acc 0.7851
[Epoch 18/200] Loss 0.9150, train acc 0.7881
[Epoch 19/200] Loss 0.8312, train acc 0.8056
Accuracy: 0.7520
[Epoch 20/200] Loss 0.7828, train acc 0.7963
[Epoch 21/200] Loss 0.7430, tra

[Epoch 175/200] Loss 0.1725, train acc 0.9456
[Epoch 176/200] Loss 0.1713, train acc 0.9520
[Epoch 177/200] Loss 0.1936, train acc 0.9385
[Epoch 178/200] Loss 0.1603, train acc 0.9555
[Epoch 179/200] Loss 0.1790, train acc 0.9479
Accuracy: 0.8200
[Epoch 180/200] Loss 0.1785, train acc 0.9485
[Epoch 181/200] Loss 0.1751, train acc 0.9479
[Epoch 182/200] Loss 0.1817, train acc 0.9467
[Epoch 183/200] Loss 0.1705, train acc 0.9502
[Epoch 184/200] Loss 0.1683, train acc 0.9467
[Epoch 185/200] Loss 0.1693, train acc 0.9467
[Epoch 186/200] Loss 0.1839, train acc 0.9444
[Epoch 187/200] Loss 0.1679, train acc 0.9532
[Epoch 188/200] Loss 0.1691, train acc 0.9496
[Epoch 189/200] Loss 0.1734, train acc 0.9461
Accuracy: 0.8200
[Epoch 190/200] Loss 0.1849, train acc 0.9397
[Epoch 191/200] Loss 0.1757, train acc 0.9432
[Epoch 192/200] Loss 0.1758, train acc 0.9444
[Epoch 193/200] Loss 0.1787, train acc 0.9461
[Epoch 194/200] Loss 0.1695, train acc 0.9432
[Epoch 195/200] Loss 0.1685, train acc 0.9514
